In [102]:
import sqlalchemy
engine = sqlalchemy.create_engine('postgresql://localhost/iet')
conn = engine.connect()

import pandas

In [115]:
def rename_tables():
    """Rename tables from gtfs_* to gtfs.*"""
    res = conn.execute(
    sqlalchemy.text(
        "select table_name from information_schema.tables where table_name like 'gtfs%'"
        )
    )
    table_names = [r[0] for r in res.fetchall()]
    for t in table_names:
        print("alter table public.{} rename to {};".format(t, t[5:]))
        print("alter table public.{} set schema gtfs;".format(t[5:]))
        
def drop_tables():
    res = conn.execute(
        sqlalchemy.text(
            "select table_name from information_schema.tables where table_schema = 'gtfs'"
        )
    )
    table_names = [r[0] for r in res.fetchall()]
    for t in table_names:
        drop_q = "drop table gtfs.{} cascade".format(t)
        print(drop_q)
        conn.execute(drop_q)

def set_timepoints(route, service, direction, seq_of_stop_ids):
    """Set timepoint = 1 on an array stops for a given route/service/direction """
    query = """
    update gtfs.stop_times 
      set timepoint = 1 
      where trip_id in 
          (select trip_id from gtfs.trips where 
              route_id = '{}' 
              and service_id = '{}' 
              and direction_id = '{}')
      and stop_id in 
    ({})    
    """.format(route, service, direction, ",".join(["'{}'".format(s) for s in seq_of_stop_ids]))
    conn.execute(query)
    return query

def get_stops(route):
    """
    Returns a DataFrame with stop sequence for a given route ID, service day, and direction.
    
    example:
    
    > # describe_trips
    > get_stop_sequence(6614)
    """
    query = """
    select
        times.stop_sequence, 
        times.arrival_time,
        trips.route_id, 
        trips.service_id,
        trips.direction_id,
        stops.stop_name,
        stops.stop_id,
        times.timepoint,
        trips.trip_id 
    from gtfs.stop_times times 
        inner join gtfs.trips trips on trips.trip_id = times.trip_id
        inner join gtfs.stops stops on stops.stop_id = times.stop_id
    where trips.trip_id in 
        (select trip_id from gtfs.trips 
            where route_id = '{}')
    order by 
        trip_id asc,
        arrival_time asc,
        stop_sequence asc;
    """.format(route)
    df = pandas.read_sql(query, conn)
    return df

In [122]:
route_id = 6640
d = get_stops(route_id)

In [123]:
# show the likely timepoints for the route
d[d.arrival_time.str.contains(":00$")].head(20)

,stop_sequence,arrival_time,route_id,service_id,direction_id,stop_name,stop_id,timepoint,trip_id
0,1,06:00:00,6640,1,0,Michigan & Schaefer,6578,1.0,1164527
5,6,06:11:00,6640,1,0,VERNOR & SALINA,10264,1.0,1164527
12,13,06:16:00,6640,1,0,Vernor & Springwells,765,1.0,1164527
15,16,06:18:00,6640,1,0,VERNOR & BEARD,7893,NaN,1164527
18,19,06:20:00,6640,1,0,Vernor & Livernois,766,1.0,1164527
27,28,06:26:00,6640,1,0,Vernor & W. Grand Blvd,767,1.0,1164527
35,36,06:33:00,6640,1,0,Trumbull - Bagley,768,1.0,1164527
39,40,06:36:00,6640,1,0,LAFAYETTE & 6TH,2621,NaN,1164527
43,44,06:41:00,6640,1,0,ROSA PARKS TRANSIT 49 EB,8915,1.0,1164527
44,1,06:25:00,6640,1,0,Michigan & Schaefer,6578,1.0,1164528


In [124]:
# show all trips with their first and last stop
d.groupby(['trip_id', 'service_id', 'direction_id'])['stop_name'].agg(['first', 'last']).head(10)

,,,first,last
trip_id,service_id,direction_id,,
1164527,1,0,Michigan & Schaefer,ROSA PARKS TRANSIT 49 EB
1164528,1,0,Michigan & Schaefer,ROSA PARKS TRANSIT 49 EB
1164529,1,0,Michigan & Schaefer,ROSA PARKS TRANSIT 49 EB
1164530,1,0,Michigan & Schaefer,ROSA PARKS TRANSIT 49 EB
1164531,1,0,Michigan & Schaefer,ROSA PARKS TRANSIT 49 EB
1164532,1,0,Michigan & Schaefer,ROSA PARKS TRANSIT 49 EB
1164533,1,0,Michigan & Schaefer,ROSA PARKS TRANSIT 49 EB
1164534,1,0,Michigan & Schaefer,ROSA PARKS TRANSIT 49 EB
1164535,1,0,Michigan & Schaefer,ROSA PARKS TRANSIT 49 EB


In [125]:
# # filter by direction and only show likely timepoints
d[d.direction_id == 1][d.service_id == '1'][d.arrival_time.str.contains(":00$")].head(5)

/home/jimmy/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


,stop_sequence,arrival_time,route_id,service_id,direction_id,stop_name,stop_id,timepoint,trip_id
1804,1,06:55:00,6640,1,1,ROSA PARKS TRANSIT 49 EB,8915,1.0,1164568
1806,3,06:57:00,6640,1,1,LAFAYETTE & WASHINGTON BLVD,1111,NaN,1164568
1815,12,07:02:00,6640,1,1,Bagley & Trumbull,770,1.0,1164568
1823,20,07:08:00,6640,1,1,Vernor & W. Grand Blvd,8774,1.0,1164568
1833,30,07:14:00,6640,1,1,Vernor & Livernois,772,1.0,1164568


In [128]:
# this is a close approximation of a schedule
df = d[d.direction_id == 1][d.service_id == '1'][d.timepoint == 1]
df = df.pivot('trip_id', 'stop_name', 'arrival_time')
df.sort_values(df.last_valid_index(), axis=1).sort_values(by=[df.columns[1]]).head()

/home/jimmy/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


stop_name,ROSA PARKS TRANSIT 49 EB,Bagley & Trumbull,Vernor & W. Grand Blvd,Vernor & Livernois,Vernor & Springwells,SALINA & VERNOR,Michigan & Schaefer
trip_id,,,,,,,
1164600,04:35:00,04:41:00,04:46:00,04:51:00,04:55:00,04:59:00,05:08:00
1164598,05:20:00,05:26:00,05:31:00,05:36:00,05:40:00,05:44:00,05:53:00
1164599,06:00:00,06:07:00,06:13:00,06:19:00,06:24:00,06:29:00,06:39:00
1164596,06:30:00,06:37:00,06:43:00,06:49:00,06:54:00,06:59:00,07:09:00
1164568,06:55:00,07:02:00,07:08:00,07:14:00,07:19:00,07:24:00,07:34:00
